This notebook will walk through the steps to make a TRK CDS image (which is not normally created as part of the MAGIC data suite) that can be used as fake track data to be used as input into the `Convert TRK postage stamp to full frame image for MAGIC.ipynb` notebook. 

In [1]:
import os

from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

from jwst_magic.fsw_file_writer import buildfgssteps, write_files, detector_effects, config

%matplotlib inline

Using backend:  Qt5Agg
Your MAGIC package is up to date
Your FGS COUNTRATE package is up to date


## 1. Run your input image through MAGIC's star selector step

Record your out directory, and other relevant information below

In [2]:
###############################################################
#### Change the parameters in this cell to match your data ####
###############################################################

out_dir = '/Users/kbrooks/git_repos/jwst-magic-fork/out/mini_otb_oct/ote29'

root = 'make_track_p100um'

guider = 1

config_number = 1

mimf = True

## 2. Create a modified config.ini file

Copy the below contents into a file called config.ini that will live in your out_dir:

```
[trk_dict]
step = 'TRK'
imgsize = 32
nramps = 5000
tframe = 0.01254
ndrops1 = 0
ndrops2 = 1
bias = true
cdsimg = true
stripsimg = false```

## 3. Run the following cells to make a TRK CDS image

In [3]:
# Path to FGS_img image - this matches the MAGIC directory structure
im = fits.getdata(os.path.join(out_dir, 'out', root, 'FGS_imgs', 
                               f'unshifted_{root}_G{guider}.fits'))
# Path to guiding selections file
guiding_selections_file = os.path.join(out_dir, 'out', root, f'guiding_config_{config_number}', 
                                       f'unshifted_guiding_selections_{root}_G{guider}_config{config_number}.txt')

# Only need a PSF center file if the input image for MIMF
if mimf: 
    psf_center_file = os.path.join(out_dir, 'out', root, 
                                   f'unshifted_psf_center_{root}_G{guider}.txt')
else:
    psf_center_file = None 

# We only care about the TRK data
step = 'TRK'

# This is the updated config.ini file mentioned above
config_file = os.path.join(out_dir, 'config.ini') 

In [4]:
bfs_obj = buildfgssteps.BuildFGSSteps(im, guider, root, step, 
                                      guiding_selections_file=guiding_selections_file, 
                                      configfile=config_file, out_dir=out_dir, 
                                      psf_center_file=psf_center_file)

2021-10-13 11:20:02,568 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_10_13_Wed_112002_buildfgssteps_make_track_p100um.log
2021-10-13 11:20:02,570 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/out/mini_otb_oct/ote29/out/make_track_p100um/guiding_config_1/unshifted_guiding_selections_make_track_p100um_G1_config1.txt as the guiding selections file
2021-10-13 11:20:02,573 - WARNING - The selected guide star triggers bright guiding. A successful threshold factor would be greater than or equal to 0.845 but the user has forced a threshold factor of 0.6
2021-10-13 11:20:04,092 - INFO - FSW File Writing: Creating TRK FSW files


In [5]:
# Update some of the object parameters to make sure the data is saved out in the correct place
bfs_obj.out_dir = os.path.join(out_dir, 'out', root, 'guiding_config_1')
bfs_obj.filename_root = '{}_G{}_{}'.format(bfs_obj.root, 
                                           bfs_obj.guider, 
                                           bfs_obj.step)

# Write out the CDS image
write_files.write_cds(bfs_obj)

2021-10-13 11:20:04,319 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/out/mini_otb_oct/ote29/out/make_track_p100um/guiding_config_1/stsci_shifted/make_track_p100um_G1_TRKcds.fits


#### About this notebook
Author: K. Brooks

Last updated: 13 October, 2021

Version of MAGIC used: 3.1.0 (commit hash: 0dbcc30)